## Apply Data Augmention

In [14]:
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
from RandomRain import RandomRain
from RandomFlare import RandomFlare
from RandomSnow import RandomSnow
from RandomFog import RandomFog
#from RandomCrop import RandomCrop
#from RandomTranslation import RandomTranslation
from RandomBlur import RandomBlur
from RandomBrightness import RandomBrightnessContrast
from HideAndSeek import HideAndSeek
from Noise import GaussianNoise
from RandomShadow import RandomShadow
from RandomGridMask import GridMask
from Segment import MeanShiftSegmentation
import getpass

def load_data(image_dir, label_dir):
    images = sorted(os.listdir(image_dir))
    labels = sorted(os.listdir(label_dir))
    assert len(images) == len(labels), "Number of images and labels must be the same"
    image_paths = [os.path.join(image_dir, img) for img in images]
    label_paths = [os.path.join(label_dir, lbl) for lbl in labels]
    return image_paths, label_paths

# Function to save augmented image and label
def save_data(output_image_dir, output_label_dir, image_name, augmented_image, label_content):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    output_image_path = os.path.join(output_image_dir, image_name)
    augmented_image.save(output_image_path)
    label_name = image_name.replace('.jpg', '.txt')
    output_label_path = os.path.join(output_label_dir, label_name)
    with open(output_label_path, 'w') as f:
        f.write(label_content)

# Function to augment images and save
def augment_and_save(image_paths, label_paths, output_image_dir, output_label_dir, num_images_to_augment, augmentations_per_image, augmentation_methods):
    available_augmenters = {
        'rain': RandomRain(),
        'flare': RandomFlare(),
        'snow': RandomSnow(),
        'fog': RandomFog(),
        #'crop': RandomCrop(),
        #'translation': RandomTranslation(),
        'blur': RandomBlur(),
        'brightness': RandomBrightnessContrast(),
        'hideandseek': HideAndSeek(),
        'noise': GaussianNoise(),
        'shadow': RandomShadow(),
        'gridmask': GridMask(),
        'segment': MeanShiftSegmentation()
    }

    selected_augmenters = {name: available_augmenters[name] for name in augmentation_methods}
    selected_indices = np.random.choice(len(image_paths), num_images_to_augment, replace=False)

    for idx in tqdm(selected_indices, desc='Augmenting images'):
        image_path = image_paths[idx]
        label_path = label_paths[idx]

        image = Image.open(image_path)
        with open(label_path, 'r') as f:
            label_content = f.read()

        for j in range(augmentations_per_image):
            # Randomly choose an augmentation technique
            augmentation_method_name = random.choice(list(selected_augmenters.keys()))
            augmentation_method = selected_augmenters[augmentation_method_name]
            # Apply augmentation
            augmented_image = augmentation_method(image)
            # Generate unique filename for augmented image
            original_filename = os.path.splitext(os.path.basename(image_path))[0]
            image_name = f'{augmentation_method_name}_{original_filename}.jpg'  # image_name = f'{augmentation_method_name}_{original_filename}_{idx:06d}_{j:03d}
            save_data(output_image_dir, output_label_dir, image_name, augmented_image, label_content)

def main():
    image_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\data(nolargebbox)\train\images"
    label_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\data(nolargebbox)\train\labels"
    output_image_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\Result data\train\images"
    output_label_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\Result data\train\labels"

    image_paths, label_paths = load_data(image_dir, label_dir)

    # User input for number of images to augment and augmentations per image
    while True:
        try:
            num_images_to_augment = int(getpass.getpass("Enter the number of images to augment: ").strip())
            if num_images_to_augment <= 0:
                print("Number of images to augment must be a positive integer.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    while True:
        try:
            augmentations_per_image = int(getpass.getpass("Enter the number of augmentations per image: ").strip())
            if augmentations_per_image <= 0:
                print("Number of augmentations per image must be a positive integer.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    augmentation_methods = getpass.getpass("Enter the augmentation methods (comma separated, available: rain, flare, snow, fog, blur, brightness, hideandseek, noise, shadow, gridmask, segment): ").strip().split(',')
    augmentation_methods = [method.strip() for method in augmentation_methods]
    augment_and_save(image_paths, label_paths, output_image_dir, output_label_dir, num_images_to_augment, augmentations_per_image, augmentation_methods)

if __name__ == "__main__":
    main()


Augmenting images: 100%|██████████| 1000/1000 [09:28<00:00,  1.76it/s]


# Balance Data

In [4]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from RandomRain import RandomRain
from RandomFlare import RandomFlare
from RandomSnow import RandomSnow
from RandomFog import RandomFog
from RandomBlur import RandomBlur
from RandomBrightness import RandomBrightnessContrast
from HideAndSeek import HideAndSeek
from Noise import GaussianNoise
from RandomShadow import RandomShadow
from RandomGridMask import GridMask
from Segment import MeanShiftSegmentation
import getpass

def load_data(image_dir, label_dir, csv_path):
    data = pd.read_csv(csv_path)
    image_paths = [os.path.join(image_dir, img) for img in data['image_name']]
    label_paths = [os.path.join(label_dir, img.replace('.jpg', '.txt')) for img in data['image_name']]
    classes = data['class'].tolist()
    return image_paths, label_paths, classes

# Function to save augmented image and label
def save_data(output_image_dir, output_label_dir, image_name, augmented_image, label_content):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    output_image_path = os.path.join(output_image_dir, image_name)
    augmented_image.save(output_image_path)
    label_name = image_name.replace('.jpg', '.txt')
    output_label_path = os.path.join(output_label_dir, label_name)
    with open(output_label_path, 'w') as f:
        f.write(label_content)

# Function to augment images and save
def augment_and_save(image_paths, label_paths, classes, output_image_dir, output_label_dir, num_images_to_augment, augmentations_per_image, augmentation_methods, class_labels):
    available_augmenters = {
        'rain': RandomRain(),
        'flare': RandomFlare(),
        'snow': RandomSnow(),
        'fog': RandomFog(),
        'blur': RandomBlur(),
        'brightness': RandomBrightnessContrast(),
        'hideandseek': HideAndSeek(),
        'noise': GaussianNoise(),
        'shadow': RandomShadow(),
        'gridmask': GridMask(),
        'segment': MeanShiftSegmentation()
    }

    selected_augmenters = {name: available_augmenters[name] for name in augmentation_methods}

    # Filter images and labels by selected classes
    filtered_data = [(image_paths[i], label_paths[i], classes[i]) for i in range(len(image_paths)) if classes[i] in class_labels]

    # Limit the number of images to augment
    filtered_data = filtered_data[:num_images_to_augment]

    existing_images = set(os.listdir(output_image_dir))
    existing_labels = set(os.listdir(output_label_dir))

    for image_path, label_path, class_id in tqdm(filtered_data, desc='Augmenting images'):
        image = Image.open(image_path)
        with open(label_path, 'r') as f:
            label_content = f.read()

        for _ in range(augmentations_per_image):
            # Randomly choose an augmentation technique
            augmentation_method_name = random.choice(list(selected_augmenters.keys()))
            augmentation_method = selected_augmenters[augmentation_method_name]
            # Apply augmentation
            augmented_image = augmentation_method(image)
            # Generate unique filename for augmented image
            original_filename = os.path.splitext(os.path.basename(image_path))[0]
            image_name = f'{augmentation_method_name}_{original_filename}.jpg'

            # Ensure no duplicate images in output directory
            if image_name not in existing_images:
                save_data(output_image_dir, output_label_dir, image_name, augmented_image, label_content)
                existing_images.add(image_name)

def main():
    image_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\data(nolargebbox)\train\images"
    label_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\data(nolargebbox)\train\labels"
    output_image_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\Result_data1\train\images"
    output_label_dir = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\Result_data1\train\labels"
    csv_path = r"C:\Users\LAPTOP\Desktop\Traffic Sign Detection With Yolo\Result_data\train.csv"  # Update with the actual path to the CSV file

    image_paths, label_paths, classes = load_data(image_dir, label_dir, csv_path)

    # User input for number of images to augment
    while True:
        try:
            num_images_to_augment = int(getpass.getpass("Enter the number of images to augment: ").strip())
            if num_images_to_augment <= 0:
                print("Number of images to augment must be a positive integer.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    # User input for number of augmentations per image
    while True:
        try:
            augmentations_per_image = int(getpass.getpass("Enter the number of augmentations per image: ").strip())
            if augmentations_per_image <= 0:
                print("Number of augmentations per image must be a positive integer.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    augmentation_methods = getpass.getpass("Enter the augmentation methods (comma separated, available: rain, flare, snow, fog, blur, brightness, hideandseek, noise, shadow, gridmask, segment): ").strip().split(',')
    augmentation_methods = [method.strip() for method in augmentation_methods]

    # User input for class labels to augment
    class_labels_input = getpass.getpass("Enter the class labels to augment (comma separated): ").strip().split(',')
    class_labels = [int(label.strip()) for label in class_labels_input]

    augment_and_save(image_paths, label_paths, classes, output_image_dir, output_label_dir, num_images_to_augment, augmentations_per_image, augmentation_methods, class_labels)

if __name__ == "__main__":
    main()


Augmenting images:   0%|          | 0/400 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\LAPTOP\\Desktop\\Traffic Sign Detection With Yolo\\data(nolargebbox)\\train\\images\\blur_00114.jpg'

: 